In [2]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import os
from skimage.io import imread, imshow
from keras import backend as K
import matplotlib.pyplot as plt
from skimage.transform import resize
from PIL import Image
import math
import argparse
import os
import time
import cv2
import gc
import random

2023-07-19 13:34:52.601999: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 13:34:52.778609: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [10]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0628/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1374.30it/s]


In [32]:
#隨機測量一張
iv = random.randint(0, len(Z_test)-1)

from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test[iv].ravel()
predition_labels = Z_test[iv].ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

acc=0.9884834289550781
iou=0.8504260800634166
predition=0.9592600871800604
recall=0.9560543611451487


In [11]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.993046765386239
iou=0.9074015868636722
predition=0.954285279869361
recall=0.948637703229964
dice=0.9514531109892875


In [34]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:37:31<00:00,  3.02s/it]

20.21095875
4.577183125
1.7907834375


In [13]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0628_skew/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1374.28it/s]


In [14]:
from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9919435643296233
iou=0.8925694225456585
predition=0.9548385651711222
recall=0.9319112320308661
dice=0.9432355948613822


In [37]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:48:24<00:00,  3.23s/it]

23.288655
5.300910625
1.93329625


In [27]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_06161/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1274.88it/s]


In [28]:
from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9930607067425915
iou=0.9071332122101518
predition=0.9590135229745477
recall=0.943720508723962
dice=0.9513055578942877


In [40]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:39:57<00:00,  3.07s/it]

20.336775
4.559755
1.76956515625


In [29]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_06162/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1277.58it/s]


In [30]:
from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / np.sum(r)
print('acc='+str(acc))

iou = (r[0][0]) / (np.sum(r)-(r[-1][-1]))
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9882618029785156
iou=0.8520790283306843
predition=0.957560369242182
recall=0.9460074571799827
dice=0.9517488554171692


In [43]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:45:16<00:00,  3.17s/it]

20.26295
4.555570625
1.7646503125


In [19]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_06164/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1376.28it/s]


In [20]:
from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.993067703794649
iou=0.9077003351001148
predition=0.9540927682204283
recall=0.9491546734388683
dice=0.9516173147314348


In [46]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:39:23<00:00,  3.06s/it]

19.94737
4.613235625
1.78454296875


In [6]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/images_andy/test/mask/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1268.87it/s]


In [8]:
from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=1.0
iou=1.0
predition=1.0
recall=1.0
dice=1.0


In [51]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:45:20<00:00,  3.17s/it]

0.0
0.0
0.0


In [21]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_061616/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1371.73it/s]


In [22]:
from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('precition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9927904112594874
iou=0.9043992659190425
predition=0.950030251521551
recall=0.9495699745478846
dice=0.9498000572716974


In [23]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:51:54<00:00,  3.30s/it]

19.97691125
4.74025125
1.83579


In [24]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 944
IMG_HEIGHT = 370
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/wrong_mask/'
TEST_PATH02 = '/Workspace/wrong_mask/no_modification_mask/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []



for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'png':

      test_mask = imread(path)
      
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'png':

      test_mask = imread(path)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 777/777 [00:00<00:00, 1003.66it/s]


In [26]:
from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('precition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9471621710055058
iou=0.49430120780708714
precition=0.6673281627705099
recall=0.6559334677880723
dice=0.6615817550364999


In [21]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((IMG_WIDTH), dtype=np.float16)
  x2=np.zeros((IMG_WIDTH*4), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(IMG_WIDTH)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - IMG_WIDTH +1)
  x_plot_d = np.arange(IMG_WIDTH) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(IMG_WIDTH)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - IMG_WIDTH + 1)
  x_plot_d_matlab=np.arange(IMG_WIDTH)+1

  eprdermis_line=np.empty(IMG_WIDTH)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(IMG_WIDTH) + 1
  y_plot_e=np.zeros(IMG_WIDTH)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(IMG_WIDTH)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(IMG_WIDTH) + 1
  y_plot_e_matlab = np.zeros(IMG_WIDTH)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(IMG_WIDTH):
    for i in range(IMG_WIDTH):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(IMG_WIDTH):
    for i in range(IMG_WIDTH):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[IMG_WIDTH -1 +k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(IMG_WIDTH):
    for i in range(IMG_WIDTH):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[IMG_WIDTH*2 -1 +k]=np.min(x1)
  for k in range(IMG_WIDTH):
    for i in range(IMG_WIDTH):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[IMG_WIDTH*3 -1 +k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[int((IMG_WIDTH*4*95)/100)]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 777/777 [1:01:49<00:00,  4.77s/it]

22.63921734234234
10.182734073359073
8.617976793758043


In [4]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0705_noise_8_Andy/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1350.02it/s]


In [5]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9928198015768519
iou=0.9047828422722594
predition=0.9501117109784376
recall=0.9499113934922587
dice=0.9500115416757147


In [6]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:39:46<00:00,  3.07s/it]

20.113255
4.68357375
1.82011234375


In [4]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0705_noise_preskew_8_Andy/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1377.38it/s]


In [5]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9928356467594304
iou=0.9049686076951687
predition=0.9503651078133252
recall=0.9498628901756652
dice=0.9501139326281024


In [6]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:50:23<00:00,  3.27s/it]

20.159965
4.639556875
1.80815578125


In [7]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0705_noise_8_Andy/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1316.77it/s]


In [8]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9928198015768519
iou=0.9047828422722594
predition=0.9501117109784376
recall=0.9499113934922587
dice=0.9500115416757147


In [9]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:37:34<00:00,  3.03s/it]

20.113255
4.68357375
1.82011234375


In [10]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0705_prenoise_skew8_Andy/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1378.02it/s]


In [11]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9921377914054444
iou=0.8949195968693192
predition=0.9571893819848654
recall=0.9322327543682574
dice=0.9445462470786155


In [12]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:45:50<00:00,  3.18s/it]

22.84980375
5.201746875
1.90355265625


In [13]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0705_10_skew8_Andy/'
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1378.97it/s]


In [14]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9921416640725808
iou=0.8946589165615948
predition=0.9601108616124056
recall=0.9291970274230067
dice=0.9444010304347671


In [15]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:39:07<00:00,  3.06s/it]

23.08456375
5.12109625
1.8823534375


In [17]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0709_noise_preskew_8_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1342.94it/s]


In [18]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.992902902610265
iou=0.905720397433456
predition=0.9518226797241165
recall=0.9492370436839144
dice=0.9505281033390229


In [19]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:48:06<00:00,  3.23s/it]

20.0785975
4.631413125
1.79801546875


In [20]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0710_noise_preskew_8_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask


Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1375.26it/s]


In [21]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9928568906141872
iou=0.9051789427143683
predition=0.9510935386339724
recall=0.9493677123019628
dice=0.9502298418485893


In [22]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:43:21<00:00,  3.14s/it]

20.091085
4.658618125
1.8146509375


In [23]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0710_noise_preskew_8_0.4_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1379.91it/s]


In [24]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9923043639333017
iou=0.8995496126667012
predition=0.9350729534647501
recall=0.9594791338134262
dice=0.9471188398220983


In [25]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:43:03<00:00,  3.13s/it]

19.85785875
5.060480625
1.95893625


In [26]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0710_noise_preskew_8_0.6_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1382.35it/s]


In [27]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9929641697499942
iou=0.9053404847251523
predition=0.9641567057727531
recall=0.9368725748021479
dice=0.9503188453540354


In [28]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:43:56<00:00,  3.15s/it]

20.92498
4.506723125
1.73624921875


In [32]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/pred_mask_testing_data_0710_noise_preskew_8_0.4_Andy/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0710_noise_preskew_8_0.4_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'png':

      test_mask = imread(path)
      
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'png':

      test_mask = imread(path)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1396.50it/s]


In [33]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=1.0
iou=1.0
predition=1.0
recall=1.0
dice=1.0


In [34]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:46:19<00:00,  3.19s/it]

0.0
0.0
0.0


In [39]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/pred_mask_testing_data_0710_noise_preskew_8_Andy/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0710_noise_preskew_8_0.6_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'png':

      test_mask = imread(path)
      
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'png':

      test_mask = imread(path)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1396.87it/s]


In [37]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9953760827636718
iou=0.9396840703882047
predition=1.0
recall=0.9396840703882047
dice=0.9689042506805121


In [38]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

  1%|█                                                                                                                                                                    | 19/3125 [00:58<2:40:23,  3.10s/it]


KeyboardInterrupt: 

In [40]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9977269091796875
iou=0.9694110299747154
predition=1.0
recall=0.9694110299747154
dice=0.9844679604411085


In [41]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:44:56<00:00,  3.17s/it]

6.810351875
1.446620625
0.46931640625


In [15]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_oct/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0705_oct_8_And/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1375.00it/s]


In [16]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9927021225449467
iou=0.9032370265273377
predition=0.9498857074770288
recall=0.9484328791891516
dice=0.9491587373910979


In [17]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0
time_start = time.time()

def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost',time_c , 's')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:45:14<00:00,  3.17s/it]

20.3638875
4.746234375
1.8406890625
time cost 9914.677531719208 s


In [18]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_oct/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0705_oct_nopretrain_8_And/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1371.84it/s]


In [19]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9928407755593345
iou=0.9048118326193545
predition=0.9526136121984939
recall=0.947455434887995
dice=0.9500275220100088


In [20]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0


def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:42:03<00:00,  3.11s/it]

20.267295
4.69409875
1.82508765625


In [3]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_oct/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0628_100_8_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1374.89it/s]


In [4]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9896862397716139
iou=0.8686313271503354
predition=0.9107386186384286
recall=0.9494633697231744
dice=0.9296979179675844


In [5]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0
time_start = time.time()

def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost',time_c , 's')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:44:06<00:00,  3.15s/it]

23.04583
6.803528125
2.4890971875
time cost 9846.054611206055 s


In [21]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_oct/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0628_10_8_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1367.97it/s]


In [22]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9917118765287436
iou=0.8909512153996759
predition=0.9418906182103941
recall=0.9427723030418224
dice=0.9423312543907828


In [23]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0
time_start = time.time()

def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost',time_c , 's')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:45:04<00:00,  3.17s/it]

21.206325
5.5007575
2.001495625
time cost 9904.800268173218 s


In [24]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_oct/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0715_noise_lr0.00001_8_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1366.90it/s]


In [25]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9927010406024652
iou=0.9033600053480676
predition=0.948544342110864
recall=0.9499099076512223
dice=0.9492266337527357


In [26]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0
time_start = time.time()

def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost',time_c , 's')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:42:12<00:00,  3.11s/it]

20.17918875
4.708145
1.82909171875
time cost 9732.73986530304 s


In [33]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0715_noise_lr0.0001_8_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1370.72it/s]


In [34]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))
#acc=0.9927219274898047
#iou=0.903654142957366
#predition=0.9483874132167492
#recall=0.9503926864376546
#dice=0.9493889909576965

acc=0.9927219274898047
iou=0.903654142957366
predition=0.9483874132167492
recall=0.9503926864376546
dice=0.9493889909576965


In [29]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0
time_start = time.time()

def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost',time_c , 's')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:45:25<00:00,  3.18s/it]

20.09597
4.71061625
1.8268778125
time cost 9925.322915792465 s


In [30]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_oct/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0715_noise_lr0.001_8_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1371.05it/s]


In [31]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9929648714179075
iou=0.9063153317361584
predition=0.9541883518548415
recall=0.9475461053484864
dice=0.950855628812197


In [32]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0
time_start = time.time()

def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost',time_c , 's')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:42:25<00:00,  3.12s/it]

20.2227625
4.623441875
1.80013359375
time cost 9745.425134181976 s


In [3]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_andy/train/'
TEST_PATH02 = '/Workspace/pred_mask_training_06168Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12500/12500 [00:09<00:00, 1315.20it/s]


In [ ]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

In [ ]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0
time_start = time.time()

def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):
        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost',time_c , 's')

In [3]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_oct/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0715_noise_lr0.004_8_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1307.94it/s]


In [3]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.986373146662583
iou=0.8130506048515386
predition=0.9823554424535189
recall=0.8251000291241922
dice=0.8968868300486462


In [6]:
#hausdorff distance
hd=0
hdsum=0
hd95=0
hd95sum=0
assd=0
assdsum=0
time_start = time.time()

def cal_distance(p1, p2):
    if p2[0] >= p1[0] and p2[1] >= p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p2[1] - p1[1])), 2))
    if p2[0] > p1[0] and p2[1] < p1[1]:
      return math.sqrt(math.pow(((p2[0] - p1[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] <= p1[0] and p2[1] <= p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p1[1] - p2[1])), 2))
    if p2[0] < p1[0] and p2[1] > p1[1]:
      return math.sqrt(math.pow(((p1[0] - p2[0])), 2) + math.pow(((p2[1] - p1[1])), 2)) 


for q in tqdm(range(len(Z_test))):
  x1=np.zeros((512), dtype=np.float16)
  x2=np.zeros((2048), dtype=np.float16)

  bri1 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)
  bri2 = np.zeros((IMG_WIDTH, 2), dtype=np.uint32)

  dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
  y_plot_d=np.empty(512)
  y_plot_d=np.argmax(dermis_line_reverse,axis=0)
  y_plot_d=abs(y_plot_d - 511)
  x_plot_d = np.arange(512) + 1


  dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
  y_plot_d_matlab=np.empty(512)
  y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
  y_plot_d_matlab=abs(y_plot_d_matlab - 511)
  x_plot_d_matlab=np.arange(512)+1

  eprdermis_line=np.empty(512)
  eprdermis_line=np.argmax(Z_test[q],axis=1) #y_plot
  x_plot_e = np.arange(512) + 1
  y_plot_e=np.zeros(512)
  y_plot_e = eprdermis_line[q]

      
  eprdermis_line_matlab=np.empty(512)
  eprdermis_line_matlab=np.argmax(Y_test[q],axis=1) #y_plot

  x_plot_e_matlab = np.arange(512) + 1
  y_plot_e_matlab = np.zeros(512)
  y_plot_e_matlab = eprdermis_line_matlab[q]

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_d_matlab[k]
          bri1[k][1] = x_plot_d_matlab[k]

  for k in range(IMG_WIDTH):        
          bri2[k][0] = y_plot_d[k]
          bri2[k][1] = x_plot_d[k]

            
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[k]=np.min(x1)
    
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[511+k]=np.min(x1)

  for k in range(IMG_WIDTH):
          bri1[k][0] = y_plot_e_matlab[k]
          bri1[k][1] = x_plot_e_matlab[k]

  for k in range(IMG_WIDTH):
    
          bri2[k][0] = y_plot_e[k]
          bri2[k][1] = x_plot_e[k]

  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri1[k], bri2[i])
    x2[1023+k]=np.min(x1)
  for k in range(512):
    for i in range(512):
      x1[i]=cal_distance (bri2[k], bri1[i])
    x2[1535+k]=np.min(x1)  

  assdsum=assdsum+np.mean(x2)
  assd=np.mean(x2)

  hdsum=hdsum+np.max(x2)
  hd=np.max(x2)

  x2=np.sort(x2)
  hd95sum=hd95sum+x2[1945]


hd=hdsum/len(Z_test)
hd95=hd95sum/len(Z_test)
assd=assdsum/len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost',time_c , 's')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [2:35:55<00:00,  2.99s/it]

33.5620575
8.942015
2.6463215625
time cost 9355.893931865692 s


In [7]:
hd = 0
hdsum = 0
hd95 = 0
assd = 0
assdsum = 0
hd95sum=0
time_start = time.time()

def cal_distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2, axis=-1))

for q in tqdm(range(len(Z_test))):
    x2=np.zeros((2048), dtype=np.float16)
    dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
    y_plot_d=np.empty(512)
    y_plot_d=np.argmax(dermis_line_reverse,axis=0)
    y_plot_d=abs(y_plot_d - 511)
    x_plot_d = np.arange(512) + 1


    dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
    y_plot_d_matlab=np.empty(512)
    y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
    y_plot_d_matlab=abs(y_plot_d_matlab - 511)
    x_plot_d_matlab=np.arange(512)+1

    eprdermis_line=np.empty(512)
    eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
    x_plot_e = np.arange(512) + 1
    y_plot_e=np.zeros(512)
    y_plot_e = eprdermis_line[q]

      
    eprdermis_line_matlab=np.empty(512)
    eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot
    x_plot_e_matlab = np.arange(512) + 1
    y_plot_e_matlab = np.zeros(512)
    y_plot_e_matlab = eprdermis_line_matlab[q]


    for k in range(512):
        x1 = cal_distance(np.array([y_plot_d_matlab[k], x_plot_d_matlab[k]]), np.stack((y_plot_d, x_plot_d), axis=-1))
        x2[k] = np.min(x1)
    for k in range(512):
        x1 = cal_distance(np.array([y_plot_d[k], x_plot_d[k]]), np.stack((y_plot_d_matlab, x_plot_d_matlab), axis=-1))
        x2[511+k] = np.min(x1)     
    for k in range(512):
        x1 = cal_distance(np.array([y_plot_e_matlab[k], x_plot_e_matlab[k]]), np.stack((y_plot_e, x_plot_e), axis=-1))
        x2[1023+k] = np.min(x1)
    for k in range(512):
        x1 = cal_distance(np.array([y_plot_e[k], x_plot_e[k]]), np.stack((y_plot_e_matlab, x_plot_e_matlab), axis=-1))
        x2[1535+k] = np.min(x1)   

    x2 = np.sort(x2)
    
    assdsum += np.mean(x2)
    hdsum += np.max(x2)
    hd95sum += x2[1945]

hd = hdsum / len(Z_test)
hd95 = hd95sum / len(Z_test)
assd = assdsum / len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost', time_c, 's')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [1:36:10<00:00,  1.85s/it]

33.5620575
17.88403
2.6463215625
time cost 5770.830108880997 s


In [9]:
hd = 0
hdsum = 0
hd95 = 0
assd = 0
assdsum = 0
hd95sum=0
time_start = time.time()

def cal_distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2, axis=-1))

for q in tqdm(range(len(Z_test))):
    x2=np.zeros((2048), dtype=np.float16)
    dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
    y_plot_d=np.empty(512)
    y_plot_d=np.argmax(dermis_line_reverse,axis=0)
    y_plot_d=abs(y_plot_d - 511)
    x_plot_d = np.arange(512) + 1


    dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
    y_plot_d_matlab=np.empty(512)
    y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
    y_plot_d_matlab=abs(y_plot_d_matlab - 511)
    x_plot_d_matlab=np.arange(512)+1

    eprdermis_line=np.empty(512)
    eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
    x_plot_e = np.arange(512) + 1
    y_plot_e=np.zeros(512)
    y_plot_e = eprdermis_line[q]

      
    eprdermis_line_matlab=np.empty(512)
    eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot
    x_plot_e_matlab = np.arange(512) + 1
    y_plot_e_matlab = np.zeros(512)
    y_plot_e_matlab = eprdermis_line_matlab[q]


    for k in range(512):
        x1 = cal_distance(np.array([y_plot_d_matlab[k], x_plot_d_matlab[k]]), np.stack((y_plot_d, x_plot_d), axis=-1))
        x2[k] = np.min(x1)

        x1 = cal_distance(np.array([y_plot_d[k], x_plot_d[k]]), np.stack((y_plot_d_matlab, x_plot_d_matlab), axis=-1))
        x2[511+k] = np.min(x1)     

        x1 = cal_distance(np.array([y_plot_e_matlab[k], x_plot_e_matlab[k]]), np.stack((y_plot_e, x_plot_e), axis=-1))
        x2[1023+k] = np.min(x1)

        x1 = cal_distance(np.array([y_plot_e[k], x_plot_e[k]]), np.stack((y_plot_e_matlab, x_plot_e_matlab), axis=-1))
        x2[1535+k] = np.min(x1)   

    x2 = np.sort(x2)
    
    assdsum += np.mean(x2)
    hdsum += np.max(x2)
    hd95sum += x2[1945]

hd = hdsum / len(Z_test)
hd95 = hd95sum / len(Z_test)
assd = assdsum / len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost', time_c, 's')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [1:36:07<00:00,  1.85s/it]

33.5866525
8.94509125
2.6469309375
time cost 5767.232126951218 s


In [11]:
import cupy as cp
hd = 0
hdsum = 0
hd95 = 0
assd = 0
assdsum = 0
hd95sum=0
time_start = time.time()

def cal_distance(p1, p2):
    return cp.sqrt(cp.sum((p1 - p2) ** 2, axis=-1))

for q in tqdm(range(len(Z_test))):
    x2=cp.zeros((2048), dtype=cp.float16)
    dermis_line_reverse=list(reversed(Z_test[q])) #這邊修改需要被測試的DATA
    y_plot_d=np.empty(512)
    y_plot_d=np.argmax(dermis_line_reverse,axis=0)
    y_plot_d=abs(y_plot_d - 511)
    x_plot_d = np.arange(512) + 1
    x_plot_d = cp.asarray(x_plot_d)
    y_plot_d = cp.asarray(y_plot_d)

    dermis_line_reverse_matlab=list(reversed(Y_test[q])) #正確組
    y_plot_d_matlab=np.empty(512)
    y_plot_d_matlab=np.argmax(dermis_line_reverse_matlab,axis=0)
    y_plot_d_matlab=abs(y_plot_d_matlab - 511)
    x_plot_d_matlab=np.arange(512)+1
    x_plot_d_matlab = cp.asarray(x_plot_d_matlab)
    y_plot_d_matlab = cp.asarray(y_plot_d_matlab)
    
    eprdermis_line=np.empty(512)
    eprdermis_line=np.argmax(Z_test,axis=1) #y_plot
    x_plot_e = np.arange(512) + 1
    y_plot_e=np.zeros(512)
    y_plot_e = eprdermis_line[q]
    x_plot_e = cp.asarray(x_plot_e)
    y_plot_e = cp.asarray(y_plot_e)
      
    eprdermis_line_matlab=np.empty(512)
    eprdermis_line_matlab=np.argmax(Y_test,axis=1) #y_plot
    x_plot_e_matlab = np.arange(512) + 1
    y_plot_e_matlab = np.zeros(512)
    y_plot_e_matlab = eprdermis_line_matlab[q]
    x_plot_e_matlab = cp.asarray(x_plot_e_matlab)
    y_plot_e_matlab = cp.asarray(y_plot_e_matlab)

    for k in range(512):
        x1 = cal_distance(cp.array([y_plot_d_matlab[k], x_plot_d_matlab[k]]), cp.stack((y_plot_d, x_plot_d), axis=-1))
        x2[k] = cp.min(x1)

        x1 = cal_distance(cp.array([y_plot_d[k], x_plot_d[k]]), cp.stack((y_plot_d_matlab, x_plot_d_matlab), axis=-1))
        x2[511+k] = np.min(x1)     

        x1 = cal_distance(cp.array([y_plot_e_matlab[k], x_plot_e_matlab[k]]), cp.stack((y_plot_e, x_plot_e), axis=-1))
        x2[1023+k] = np.min(x1)

        x1 = cal_distance(cp.array([y_plot_e[k], x_plot_e[k]]), cp.stack((y_plot_e_matlab, x_plot_e_matlab), axis=-1))
        x2[1535+k] = np.min(x1)   

    x2 = cp.sort(x2)
    
    assdsum += cp.mean(x2)
    hdsum += cp.max(x2)
    hd95sum += x2[1945]

hd = hdsum / len(Z_test)
hd95 = hd95sum / len(Z_test)
assd = assdsum / len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost', time_c, 's')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [1:52:05<00:00,  2.15s/it]

inf
8.6016
2.65216
time cost 6725.298031806946 s


In [22]:
import cupy as cp

hd = 0
hdsum = 0
hd95 = 0
assd = 0
assdsum = 0
hd95sum=0
time_start = time.time()

def cal_distance(p1, p2):
    return cp.sqrt(cp.sum((p1 - p2) ** 2, axis=-1))
z_test = cp.asarray(Z_test)
y_test = cp.asarray(Y_test)
for q in tqdm(range(len(Z_test))):

    x2=cp.zeros((2048), dtype=cp.float32)
    x1=cp.zeros((512), dtype=cp.float32)
    dermis_line_reverse=cp.flip(z_test[q]) #這邊修改需要被測試的DATA
    y_plot_d = cp.argmax(dermis_line_reverse,axis=0)
    y_plot_d =abs(y_plot_d - 511)
    x_plot_d = cp.arange(512) + 1


    dermis_line_reverse_matlab=cp.flip(y_test[q]) #正確組
    y_plot_d_matlab=cp.argmax(dermis_line_reverse_matlab,axis=0)
    y_plot_d_matlab=abs(y_plot_d_matlab - 511)
    x_plot_d_matlab=cp.arange(512)+1

    
    eprdermis_line=cp.argmax(z_test,axis=1) #y_plot
    x_plot_e = cp.arange(512) + 1
    y_plot_e=cp.zeros(512)
    y_plot_e = eprdermis_line[q]

      
    eprdermis_line_matlab=cp.empty(512)
    eprdermis_line_matlab=cp.argmax(y_test,axis=1) #y_plot
    x_plot_e_matlab = cp.arange(512) + 1
    y_plot_e_matlab = cp.zeros(512)
    y_plot_e_matlab = eprdermis_line_matlab[q]


    for k in range(512):
        x1 = cal_distance(cp.array([y_plot_d_matlab[k], x_plot_d_matlab[k]]), cp.stack((y_plot_d, x_plot_d), axis=-1))
        x2[k] = cp.min(x1)

        x1 = cal_distance(cp.array([y_plot_d[k], x_plot_d[k]]), cp.stack((y_plot_d_matlab, x_plot_d_matlab), axis=-1))
        x2[511+k] = np.min(x1)     

        x1 = cal_distance(cp.array([y_plot_e_matlab[k], x_plot_e_matlab[k]]), cp.stack((y_plot_e, x_plot_e), axis=-1))
        x2[1023+k] = np.min(x1)

        x1 = cal_distance(cp.array([y_plot_e[k], x_plot_e[k]]), cp.stack((y_plot_e_matlab, x_plot_e_matlab), axis=-1))
        x2[1535+k] = np.min(x1) 

    x2 = cp.sort(x2)
    
    assdsum += cp.mean(x2)
    hdsum += cp.max(x2)
    hd95sum += x2[1945]

hd = hdsum / len(Z_test)
hd95 = hd95sum / len(Z_test)
assd = assdsum / len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost', time_c, 's')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [18:35<00:00,  2.80it/s]

33.58996
8.947334
2.6472716
time cost 1116.5390100479126 s


In [25]:
seed = 46
np.random.seed = seed

IMG_WIDTH = 512
IMG_HEIGHT = 512
#IMG_CHANNELS = 1 


TEST_PATH = '/Workspace/images_oct/test/'
TEST_PATH02 = '/Workspace/pred_mask_testing_data_0715_noise_lr0.00004_8_Andy/' 
test_ids = next(os.walk(TEST_PATH))[2]

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
Z_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint16)
sizes_test = []

print('Resizing test images') 
for n ,id_ in enumerate(test_ids):
    path =TEST_PATH + id_
    img = imread(path)[:,:]
    #image =imread(path + '/images/' +id_ + '.pgm')[:,:] 
    sizes_test.append([img.shape[0], img.shape[1]])
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img 

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH)]
      name2=path[len(TEST_PATH):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/4
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'
      img = imread(path)[:,:]  
        
      X_test[n] = img  #Fill empty X_train with values from img
    
      mpath = name1 + "mask/" + name2 + m2[:-2] + '.png'
      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Y_test[n] = test_mask
    
    
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):   
    path = TEST_PATH02 + id_ 
    if (path[-3::]) == 'pgm':
      name1=path[:len(TEST_PATH02)]
      name2=path[len(TEST_PATH02):-8]
      m1=path[-8:-4]
      mth=int(m1)
      mth=mth/1
      m2=str(mth)
      while len(m2)<6:
        m2='0'+m2
      m3=path[:-8] + m2[:-2] + '.png'

      mpath =  m3

      test_mask = imread(mpath)
      test_mask=np.where(test_mask<128, 0, test_mask)
      Z_test[n] = test_mask

Resizing test images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:02<00:00, 1377.16it/s]


In [26]:
#測試組的全部


from sklearn.metrics import confusion_matrix

ground_truth_labels = Y_test.ravel()
predition_labels = Z_test.ravel()

r = confusion_matrix(y_true=ground_truth_labels, y_pred=predition_labels)
r = np.flip(r)

acc = (r[0][0] + r[-1][-1]) / (r[0][0] + r[0][-1] + r[-1][0]+ r[-1][-1])
print('acc='+str(acc))

iou = r[0][0] / (r[0][0] + r[0][-1] + r[-1][0])
print('iou='+ str(iou))

precision = r[0][0] / (r[0][0] + r[-1][0])
print('predition='+str(precision))

recall = r[0][0] / (r[0][0] + r[0][-1])
print('recall='+str(recall))

dice = 2*(precision*recall/(recall+precision))
print('dice='+str(dice))

acc=0.9927109608619304
iou=0.9034716778676302
predition=0.9487434838984339
recall=0.949833702792546
dice=0.9492882803276033


In [29]:
import cupy as cp

hd = 0
hdsum = 0
hd95 = 0
assd = 0
assdsum = 0
hd95sum=0
time_start = time.time()

def cal_distance(p1, p2):
    return cp.sqrt(cp.sum((p1 - p2) ** 2, axis=-1))
z_test = cp.asarray(Z_test)
y_test = cp.asarray(Y_test)
for q in tqdm(range(len(Z_test))):

    x2=cp.zeros((2048), dtype=cp.float32)
    x1=cp.zeros((512), dtype=cp.float32)
    dermis_line_reverse=cp.flip(z_test[q]) #這邊修改需要被測試的DATA
    y_plot_d = cp.argmax(dermis_line_reverse,axis=0)
    y_plot_d =abs(y_plot_d - 511)
    x_plot_d = cp.arange(512) + 1


    dermis_line_reverse_matlab=cp.flip(y_test[q]) #正確組
    y_plot_d_matlab=cp.argmax(dermis_line_reverse_matlab,axis=0)
    y_plot_d_matlab=abs(y_plot_d_matlab - 511)
    x_plot_d_matlab=cp.arange(512)+1

    
    eprdermis_line=cp.argmax(z_test,axis=1) #y_plot
    x_plot_e = cp.arange(512) + 1
    y_plot_e=cp.zeros(512)
    y_plot_e = eprdermis_line[q]

      
    eprdermis_line_matlab=cp.empty(512)
    eprdermis_line_matlab=cp.argmax(y_test,axis=1) #y_plot
    x_plot_e_matlab = cp.arange(512) + 1
    y_plot_e_matlab = cp.zeros(512)
    y_plot_e_matlab = eprdermis_line_matlab[q]


    for k in range(512):
        x1 = cal_distance(cp.array([y_plot_d_matlab[k], x_plot_d_matlab[k]]), cp.stack((y_plot_d, x_plot_d), axis=-1))
        x2[k] = cp.min(x1)

        x1 = cal_distance(cp.array([y_plot_d[k], x_plot_d[k]]), cp.stack((y_plot_d_matlab, x_plot_d_matlab), axis=-1))
        x2[511+k] = cp.min(x1)     

        x1 = cal_distance(cp.array([y_plot_e_matlab[k], x_plot_e_matlab[k]]), cp.stack((y_plot_e, x_plot_e), axis=-1))
        x2[1023+k] = cp.min(x1)

        x1 = cal_distance(cp.array([y_plot_e[k], x_plot_e[k]]), cp.stack((y_plot_e_matlab, x_plot_e_matlab), axis=-1))
        x2[1535+k] = cp.min(x1) 

    x2 = cp.sort(x2)
    
    assdsum += cp.mean(x2)
    hdsum += cp.max(x2)
    hd95sum += x2[1945]

hd = hdsum / len(Z_test)
hd95 = hd95sum / len(Z_test)
assd = assdsum / len(Z_test)

print(hd)
print(hd95)
print(assd)
time_end = time.time()
time_c = time_end - time_start
print('time cost', time_c, 's')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [18:12<00:00,  2.86it/s]

20.173346
4.7046313
1.8282905
time cost 1092.9366462230682 s
